## Teams
In Autogen a group of two or more agents  working together is called a `Team`.  

AutoGen offers differents pre-made agent workflows, let's see the simplest one called `RoundRobinGroupChat`

### RoundRobinGroupChat
Its a simple workflow where all partecipant agents share the same context and take turns responding in a round-robin fashion.

In [1]:
import sys
sys.path.append("..")
from model_clients.azure import get_model
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from rich import print


In the following cell we create the two agents that will make up the team, one is in charge of writing a poem, the other to evaluate it.

In [2]:
# Create the primary agent.
writer_agent = AssistantAgent(
    "writer",
    model_client=get_model(),
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=get_model(),
    system_message="Provide constructive feedback. Respond with 'APPROVE' only when your feedbacks are addressed.",
)

We now put the two individual agent in contact with each other using a `RoundRobinGroupChat` that let execution flow go from 
the writer agent to the critic agent and then back to writer agent and so on.  

Of course we need a way to stop this ping-pong that's why we create a `text_termination` rule, in this case the rule will be
triggered when the word **APPROVE** will appear in the agent response.

In [3]:
# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the two agents and the termination condition.
team = RoundRobinGroupChat(
    [writer_agent, critic_agent], termination_condition=text_termination
)

It's now time to run the Team using the `run` method.

In [4]:
result:TaskResult = await team.run(task="Write a short poem about the fall season.")
print(result)

TaskResult(
    messages=[
        TextMessage(
            source='user',
            models_usage=None,
            metadata={},
            content='Write a short poem about the fall season.',
            type='TextMessage'
        ),
        TextMessage(
            source='writer',
            models_usage=RequestUsage(prompt_tokens=28, completion_tokens=106),
            metadata={},
            content="Golden leaves cascade from trees,  \nWhispering secrets with the breeze.  \nCrisp air dances 
on fingertips,  \nAs autumn's paintbrush softly dips.\n\nPumpkins sit with smiling beams,  \nWhile cider warms the 
sweetest dreams.  \nThe world adorned in hues of flame,  \nIn fall's embrace, we're not the same.\n\nNature dons 
its vibrant robe,  \nAs twilight glows across the globe.  \nIn fall's brief kiss, the heart's unbound,  \nWhere 
beauty in each moment's found.",
            type='TextMessage'
        ),
        TextMessage(
            source='critic',
            models_usage=RequestUsage(prompt_tokens=151, completion_tokens=223),
            metadata={},
            content='Your poem beautifully captures the essence of fall with vivid imagery and a warm, nostalgic 
tone. Here are a few suggestions to enhance it even further:\n\n1. The use of "golden" and "cascade" exemplifies 
movement and color, but consider adding another sensory detail or metaphor to deepen the scene.\n   - Example: 
"Golden leaves cascade like whispered songs in the twilight breeze."\n\n2. You have a lovely thematic consistency 
with "cider warming dreams" and "hues of flame." You could extend this imagery into the final stanza to strengthen 
unity.\n   - Example: "Twilight\'s glow wraps us in flame, in fall\'s brief kiss, we\'re not the same."\n\n3. 
Consider a slight rhythmic enhancement to achieve a smoother flow and lyrical quality.\n   - Example: "As autumn\'s
paintbrush softly tips."\n\nOverall, your poem wonderfully evokes the magic of fall. With some minor adjustments to
flow and imagery, it can sparkle even more effectively. \n\nOnce these thoughts are considered, feel free to 
revise, or if you believe the initial version suits your intention best, let me know. ',
            type='TextMessage'
        ),
        TextMessage(
            source='writer',
            models_usage=RequestUsage(prompt_tokens=367, completion_tokens=134),
            metadata={},
            content="Thank you for your thoughtful feedback and suggestions. Here's a revised version incorporating
your ideas to enhance the imagery and rhythmic flow:\n\nGolden leaves cascade like whispered songs,  \nAdorning 
trees with autumn's vibrant throngs.  \nCrisp air dances on fingertips,  \nAs nature's paintbrush softly 
tips.\n\nPumpkins sit with smiling beams,  \nWhile cider warms our sweetest dreams.  \nThe world adorned in hues of
flame,  \nIn twilight's glow, we're not the same.\n\nNature dons its vibrant robe,  \nAs twilight wraps us 'round 
the globe.  \nIn fall's brief kiss, the heart's unbound,  \nWhere beauty in each moment's found.",
            type='TextMessage'
        ),
        TextMessage(
            source='critic',
            models_usage=RequestUsage(prompt_tokens=518, completion_tokens=106),
            metadata={},
            content='Your revised poem has successfully incorporated more sensory details, enhancing both the 
imagery and rhythmic flow. The addition of "like whispered songs" adds a lyrical quality that deepens the portrayal
of fall. The adjustments in phrasing create a smoother and more engaging rhythm, making the poem even more 
captivating. The thematic consistency with “twilight\'s glow” provides a cohesive and unified feel to the entire 
piece. \n\nOverall, your revisions have beautifully enhanced the poem while maintaining its original warmth and 
evocative imagery. Well done!\n\nAPPROVE',
            type='TextMessage'
        )
    ],
    stop_reason="Text 'APPROVE' mentioned"
)

We can also use Autogen's `Console`class to see a more structured output.  

Note that a Team (as Agents) keep their state, so to start a new run from scratch it is usually a good idea
 to reset the team if the next task is not related to the previous task with the `reset` method.

In [5]:
await team.reset()  # Reset the team for a new task.
await Console(
    team.run_stream(task="Write a joke about programming in Python.")
)  

---------- user ----------
Write a joke about programming in Python.
---------- writer ----------
Why did the Python programmer break up with their string?

It wasn't her type!
---------- critic ----------
This is a clever joke that utilizes programming terminology effectively. It plays on the concept of "types" in programming languages and relates it to a personal relationship, which is relatable and humorous. However, you might want to consider adding a bit more context for those unfamiliar with programming terms to make it accessible to a wider audience. Here's a suggestion:

Why did the Python programmer break up with their string?

They realized she wasn't their type—literally! (In Python, strings have a specific 'type' just like integers and floats.)

Feel free to revise or expand upon this idea for broader engagement. 

Please make changes based on this feedback for approval.
---------- writer ----------
Why did the Python programmer break up with their string?

They realized sh

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a joke about programming in Python.', type='TextMessage'), TextMessage(source='writer', models_usage=RequestUsage(prompt_tokens=27, completion_tokens=17), metadata={}, content="Why did the Python programmer break up with their string?\n\nIt wasn't her type!", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=61, completion_tokens=131), metadata={}, content='This is a clever joke that utilizes programming terminology effectively. It plays on the concept of "types" in programming languages and relates it to a personal relationship, which is relatable and humorous. However, you might want to consider adding a bit more context for those unfamiliar with programming terms to make it accessible to a wider audience. Here\'s a suggestion:\n\nWhy did the Python programmer break up with their string?\n\nThey realized she wasn\'t their type—literally! (In Python, s

Together with `TextTermination`is it possible to interrupt a team any time using [ExternalTermination](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/teams.html#stopping-a-team) 
or one of the [other](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/termination.html#termination) termination conditions.